In [11]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import cmasher as cmr
import scipy
mpl.rcParams['hatch.linewidth'] = 0.5
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import warnings
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import numba
import statsmodels.api as sm
import glob
import pandas as pd

In [4]:
# Open SST data 
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

In [21]:
# Take the weighted area average
nino_34_weights = np.cos(np.deg2rad(sst_nino_34.latitude))

sst_nino_34_mean = sst_nino_34.weighted(nino_34_weights).mean(dim=('latitude', 'longitude'))

In [115]:
sst_anoms = np.zeros((1668))
for i in range(26):
    if i < 25:
        # Compute SST climo 
        climo = sst_nino_34_mean.sel(time=sst_nino_34_mean.time.dt.year.isin(np.arange(base_period_start_years[i], base_period_end_years[i]+1, 1))).groupby('time.month').mean()
    
        # Subset years of interest
        yrs_of_int = sst_nino_34_mean.sel(time=sst_nino_34_mean.time.dt.year.isin(np.arange(test_period_start_years[i], test_period_end_years[i]+1, 1)))
    
        # Compute anomalies and store
        sst_anoms[i*60:(i+1)*60] = yrs_of_int.groupby('time.month') - climo
        
        # Prove that you used the right base period and test period
        # print(f'base period: {base_period_start_years[i]}-{base_period_end_years[i]}, test period: {test_period_start_years[i]}-{test_period_end_years[i]}')
    else: 
        # Same climo as before (1991-2020)
        climo = sst_nino_34_mean.sel(time=sst_nino_34_mean.time.dt.year.isin(np.arange(base_period_start_years[24], base_period_end_years[24]+1, 1))).groupby('time.month').mean()

        # Years of interest
        yrs_of_int = sst_nino_34_mean.sel(time=sst_nino_34_mean.time.dt.year.isin(np.arange(2011, 2025, 1)))

        # Compute anomalies and store
        sst_anoms[1500:] = yrs_of_int.groupby('time.month') - climo

In [116]:
# Convert back to XR
sst_anoms_xr = xr.DataArray(sst_anoms, coords={'time':sst['time'][12*16:]}, dims=['time'])

In [118]:
# Take the three month rolling mean
sst_anoms_rolling = sst_anoms_xr.rolling(time=3, center=True).mean()

In [122]:
sst_anoms_rolling.sel(time=sst_anoms_rolling.time.dt.year.isin(np.arange(1950, 2025, 1)))[0:12]

<xarray.DataArray (time: 12)> Size: 96B
array([-1.19203186, -1.1771698 , -1.15901756, -1.04081853, -0.92330742,
       -0.88340759, -0.63889186, -0.69064267, -0.59530258, -0.81159147,
       -0.8365771 , -1.00986989])
Coordinates:
  * time     (time) datetime64[ns] 96B 1950-01-16T12:00:00 ... 1950-12-16T12:...

In [124]:
def oni_moving_base(sst_data):
    ## Assumes that the time period of analysis is 1870-2024
    ## Returns the monthly ONI index from 1885-2024
    
    # Subset nino_34 region
    nino_34 = sst_data.sel(longitude=slice(-170, -120)).sel(latitude=slice(5, -5))

    # Take the weighted area average
    weights = np.cos(np.deg2rad(nino_34.latitude))

    nino_34_mean = nino_34.weighted(weights).mean(dim=('latitude', 'longitude'))

    # Define base period and test period start and end years
    base_period_start_years = np.arange(1871, 1996, 5)
    base_period_end_years = np.arange(1900, 2021, 5)

    test_period_start_years = np.arange(1886, 2011, 5)
    test_period_end_years = np.arange(1890, 2014, 5)

    # Compute monthly SST anoms
    sst_anoms = np.zeros((1668))
    for i in range(26):
        if i < 25:
            # Compute SST climo 
            climo = nino_34_mean.sel(time=nino_34_mean.time.dt.year.isin(np.arange(base_period_start_years[i], base_period_end_years[i]+1, 1))).groupby('time.month').mean()
        
            # Subset years of interest
            yrs_of_int = nino_34_mean.sel(time=nino_34_mean.time.dt.year.isin(np.arange(test_period_start_years[i], test_period_end_years[i]+1, 1)))
        
            # Compute anomalies and store
            sst_anoms[i*60:(i+1)*60] = yrs_of_int.groupby('time.month') - climo
            
            # Prove that you used the right base period and test period
            # print(f'base period: {base_period_start_years[i]}-{base_period_end_years[i]}, test period: {test_period_start_years[i]}-{test_period_end_years[i]}')
        else: 
            # Same climo as before (1991-2020)
            climo = nino_34_mean.sel(time=nino_34_mean.time.dt.year.isin(np.arange(base_period_start_years[24], base_period_end_years[24]+1, 1))).groupby('time.month').mean()
    
            # Years of interest
            yrs_of_int = nino_34_mean.sel(time=nino_34_mean.time.dt.year.isin(np.arange(2011, 2025, 1)))
    
            # Compute anomalies and store
            sst_anoms[1500:] = yrs_of_int.groupby('time.month') - climo

    # Convert sst_anoms back to XR
    sst_anoms_xr = xr.DataArray(sst_anoms, coords={'time':sst_data['time'][12*16:]}, dims=['time'])

    return sst_anoms_xr

In [125]:
test = oni_moving_base(sst)